In [1]:
pip install accelerate -U

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
pip install transformers[torch]

#### Import Libraries

In [1]:
import zipfile

In [2]:
with zipfile.ZipFile('/content/drive/MyDrive/nlp-getting-started.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [3]:
import torch

In [4]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from textblob import Word
import string
from transformers import BertTokenizer, BertForSequenceClassification

In [5]:
from torch.utils.data import Dataset, DataLoader

#### Load Data

In [6]:
Df = pd.read_csv('/content/train.csv')

In [7]:
DfTest = pd.read_csv('/content/test.csv')

In [8]:
Dftext = Df['text']
DfY = Df['target']

In [9]:
DfTesttext = DfTest['text']
DfTestid = DfTest['id']

#### Load the tokeniser for the Bert model

In [10]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

#### Load the pretrained Bert model

In [11]:
model =  AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
train_text = Dftext.tolist()
y_train = DfY.tolist()

##### Tokenise the text with the loaded tokeniser

In [14]:
tokenised_textB = tokenizer(train_text, padding=True, truncation=True)

##### Optimizer

In [15]:
from transformers import AdamW
optimiser = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### Process Data to make it compatible for model training

In [16]:
class TrainDataset(Dataset):
  def __init__(self, data, labels):
    self.data = data
    self.labels = labels
    assert len(self.data['input_ids']) == len(self.labels)
  def __len__(self):
    return len(self.labels)
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

In [17]:
train_dataset = TrainDataset(tokenised_textB, DfY.tolist())

In [18]:
Train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

####Training the model

In [19]:
from transformers import Trainer,TrainingArguments

In [20]:
from transformers import get_linear_schedule_with_warmup

In [21]:
len(train_dataset)

7613

In [23]:
trainargs = TrainingArguments(output_dir='./results',
    num_train_epochs=3,
    warmup_steps=500,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
)


In [28]:
num_training_steps = len(train_dataset) // trainargs.per_device_train_batch_size * trainargs.num_train_epochs

In [29]:
trn_scheduler = get_linear_schedule_with_warmup(optimiser, num_warmup_steps=500, num_training_steps=-1)

In [30]:
trner = Trainer(
    model=model,
    args=trainargs,
    train_dataset=train_dataset,
    optimizers=(optimiser,trn_scheduler),

)

In [31]:
trner.train()

Step,Training Loss
100,0.683400
200,0.498200
300,0.387800
400,0.447800
500,0.429000
600,0.353600
700,0.368300
800,0.363800
900,0.345800
1000,0.343300


TrainOutput(global_step=1428, training_loss=0.4030073316825204, metrics={'train_runtime': 365.406, 'train_samples_per_second': 62.503, 'train_steps_per_second': 3.908, 'total_flos': 985883291099280.0, 'train_loss': 0.4030073316825204, 'epoch': 3.0})

In [ ]:
max_length = 84

#### Prepare the test data for prediction

In [32]:
Tokenised_textT = tokenizer(DfTesttext.tolist(), padding=True, truncation=True, return_tensors='pt')

In [34]:
class testDataset(Dataset):
  def __init__(self, data):
    self.data = data
  def __len__(self):
    return len(self.data['input_ids'])
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}
    return item

In [35]:
DatasetT = testDataset(Tokenised_textT)

In [36]:
DataloaderT = DataLoader(DatasetT, batch_size=32, shuffle=False)

#### Switch model to evaluation mode

In [37]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [38]:
predictions =[]

In [39]:
with torch.no_grad():
    for batch in DataloaderT:
        inputs = {key: val.to(model.device) for key, val in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predictions.append(logits)

<ipython-input-34-f572f0b577f7>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}


##### We receive the predictions as batches so we ungroup them

In [40]:
predictions = torch.cat(predictions)

In [43]:
predictions.shape

torch.Size([3263, 2])

##### Convert the prediction to single file

In [44]:
import numpy as np

In [45]:
predicted_labels = torch.argmax(predictions, dim=1)

In [46]:
predicted_labels.shape

torch.Size([3263])

In [48]:
#The tensors are stored in the gpus so we switch them to the cpu
t = predicted_labels.cpu()

In [49]:
# prompt: convert predicted_labels to dataframe with column 'target'

df_predicted = pd.DataFrame(t, columns=['target'])


In [50]:
df_predicted['id'] = DfTestid

In [51]:
df_predicted = df_predicted[['id','target']]

In [52]:
df_predicted

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [ ]:
df_predicted.to_csv('/content/submitBertTrnsch.csv',index=False)